In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge requests --yes
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library



print('Libraries imported.')

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

# A practice using Foursquare API's location data to check coffee shops in Shanghai geographical center.

## Define the centroid needed for searching

In [7]:
address = '168 Zhoujin Rd, Huangpu, Shanghai, China'                 # A Costa Coffee (咖世家) located in [168 Zhoujin Rd | 昼锦路168号]

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of our centroid are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of our centroid are 31.226294, 121.4869945.


## Define geographical info needed

In [8]:
# create map of Shanghai using latitude and longitude values
map_Shanghai = folium.Map(location=[latitude, longitude], zoom_start=10)

In [4]:
map_Shanghai

## Define our search criteria

In [10]:
search_query = 'Coffee'
radius = 1000
print(search_query + ' .... OK!')

Coffee .... OK!


## Define 4square client info

In [4]:
CLIENT_ID = "*************************************************"                              # Sry, ommitted                        
CLIENT_SECRET = "**********************************************"                            # Sry, ommitted again
VERSION = "20190520"
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: *************************************************
CLIENT_SECRET:**********************************************


## Define URL and get JSON file

In [12]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},\
{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [13]:
results = requests.get(url).json()

## assign relevant part of JSON to venues

In [14]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()
dataframe.count()

categories                   43
hasPerk                      43
id                           43
location.address             25
location.cc                  43
location.city                39
location.country             43
location.crossStreet         13
location.distance            43
location.formattedAddress    43
location.labeledLatLngs      43
location.lat                 43
location.lng                 43
location.neighborhood         3
location.postalCode           5
location.state               39
name                         43
referralId                   43
dtype: int64

## Keep only columns that include venue name, location associated fields

In [15]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Kafelaku Coffee (猫屎咖啡),Café,NaN,CN,上海市,中国,NaN,139,"[上海市, 上海市, 中国]","[{'label': 'display', 'lat': 31.22713085887843...",31.227131,121.488080,NaN,NaN,上海市,5219c6b1498ee106267e3eae
1,Costa Coffee (咖世家),Coffee Shop,168 Zhoujin Rd | 昼锦路168号,CN,Huangpu,中国,Yu Fashion Garden | 豫城时尚,98,[168 Zhoujin Rd | 昼锦路168号 (Yu Fashion Garden |...,"[{'label': 'display', 'lat': 31.22684288569973...",31.226843,121.487807,Yùyuán,200085,上海市,50b70a92e4b055ce10fff6bf
2,J.co Donuts & Coffee (杰酷),Donut Shop,168 Zhoujin Rd | 昼锦路168号,CN,Huangpu,中国,1/F Yu Fashion Garden | 豫城时尚,159,[168 Zhoujin Rd | 昼锦路168号 (1/F Yu Fashion Gard...,"[{'label': 'display', 'lat': 31.227139, 'lng':...",31.227139,121.488342,Yùyuán,200085,上海市,4e1045b77d8b4d5613d0561d
3,Costa Coffee (咖世家),Coffee Shop,NaN,CN,上海市,中国,NaN,335,"[上海市, 上海市, 中国]","[{'label': 'display', 'lat': 31.22855603526196...",31.228556,121.484672,NaN,NaN,上海市,4d3263195017a0934e08499b
4,Gucchio Coffee (古墙咖啡),Coffee Shop,Renmin Road- Near HuaiHai Zhong Road,CN,上海市,中国,Lanson Place Basement Floor,280,[Renmin Road- Near HuaiHai Zhong Road (Lanson ...,"[{'label': 'display', 'lat': 31.22698, 'lng': ...",31.226980,121.484162,NaN,NaN,上海市,59abd0e09ef8ef7b9d44c882
5,Pacific Coffee (太平洋咖啡),Coffee Shop,NaN,CN,上海市,中国,NaN,1061,"[上海市, 上海市, 中国]","[{'label': 'display', 'lat': 31.23473194944199...",31.234732,121.481800,NaN,NaN,上海市,55bab71c498e59a1e27b5a81
6,Pacific Coffee (太平洋咖啡),Coffee Shop,Liuling Rd,CN,上海市,中国,1/F Times Square,1099,"[Liuling Rd (1/F Times Square), 上海市, 上海市, 中国]","[{'label': 'display', 'lat': 31.22843830958441...",31.228438,121.475720,NaN,NaN,上海市,4ed0efedb634dd29921d57d6
7,Pacific Coffee (太平洋咖啡),Coffee Shop,Huaihai (M) Rd,CN,新莊區,中国,"Room 02, 1/F Lansheng Building",1113,"[Huaihai (M) Rd (Room 02, 1/F Lansheng Buildin...","[{'label': 'display', 'lat': 31.2280714056927,...",31.228071,121.475479,NaN,NaN,上海市,4de1d2aad164df8574ed7c40
8,Costa Coffee (咖世家),Coffee Shop,666 Fuzhou Rd,CN,Huangpu,中国,NaN,1311,"[666 Fuzhou Rd, Huangpu, 上海市, 中国]","[{'label': 'display', 'lat': 31.23112, 'lng': ...",31.231120,121.474427,NaN,NaN,上海市,56b54290498e3f17cbf218e8
9,憨豆咖啡 Mr. Bean Coffee,Coffee Shop,河南南路489号香港名都一楼,CN,Huangpu,中国,复兴东路,254,"[河南南路489号香港名都一楼 (复兴东路), Huangpu, 上海市, 中国]","[{'label': 'display', 'lat': 31.22484809856239...",31.224848,121.484918,NaN,NaN,上海市,4d05f38f30a58cfa7819a8e7


In [17]:
dataframe_filtered.name

0               Kafelaku Coffee (猫屎咖啡)
1                   Costa Coffee (咖世家)
2            J.co Donuts & Coffee (杰酷)
3                   Costa Coffee (咖世家)
4                Gucchio Coffee (古墙咖啡)
5               Pacific Coffee (太平洋咖啡)
6               Pacific Coffee (太平洋咖啡)
7               Pacific Coffee (太平洋咖啡)
8                   Costa Coffee (咖世家)
9                 憨豆咖啡 Mr. Bean Coffee
10                 Mr Bean Coffee Shop
11                      Bin Dao Coffee
12                       Coffee Midori
13                    Java Coffee Shop
14                       Manner Coffee
15                        Coffee House
16                         More Coffee
17    The Coffee Bean & Tea Leaf (香啡缤)
18                  Luckin Coffee 瑞幸咖啡
19                       Seesaw Coffee
20                   UBC Coffee (上岛咖啡)
21                      Maancat Coffee
22                       Manner Coffee
23                          Zoo Coffee
24                  Sundeck Coffee Bar
25                       

In [18]:
dataframe_filtered.count()

name                43
categories          42
address             25
cc                  43
city                39
country             43
crossStreet         13
distance            43
formattedAddress    43
labeledLatLngs      43
lat                 43
lng                 43
neighborhood         3
postalCode           5
state               39
id                  43
dtype: int64

In [19]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the geographical center of SH
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Geographical Center of SH',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the coffee shops as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.3
    ).add_to(venues_map)

# display map
venues_map

## Check ratings of several coffee shops

In [20]:
venue_id = '5219c6b1498ee106267e3eae' # Kafelaku Coffee (猫屎咖啡)
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/5219c6b1498ee106267e3eae?client_id=JRMAOQ23QH2AR2G01QNCFARI5IKTISMVT2PHM3D0DXKGHF5N&client_secret=M1P2CGJKPR0J1N1GMUV0P5CH1BGLFD3SHIPP4YTOFQ4NTATE&v=20190520'

In [21]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])


{'id': '5219c6b1498ee106267e3eae',
 'name': 'Kafelaku Coffee (猫屎咖啡)',
 'contact': {},
 'location': {'lat': 31.22713085887843,
  'lng': 121.48808037173434,
  'labeledLatLngs': [{'label': 'display',
    'lat': 31.22713085887843,
    'lng': 121.48808037173434}],
  'cc': 'CN',
  'city': '上海市',
  'state': '上海市',
  'country': '中国',
  'formattedAddress': ['上海市', '上海市', '中国']},
 'canonicalUrl': 'https://foursquare.com/v/kafelaku-coffee-%E7%8C%AB%E5%B1%8E%E5%92%96%E5%95%A1/5219c6b1498ee106267e3eae',
 'categories': [{'id': '4bf58dd8d48988d16d941735',
   'name': 'Café',
   'pluralName': 'Cafés',
   'shortName': 'Café',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '¥'},
 'likes': {'count': 1,
  'groups': [{'type': 'others',
    'count': 1,
    'items': [{'id': '874932',
      'firstName': 'Alex',
      'lastName': 'Llamas',


In [22]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [23]:
venue_id = '4bf2301b189f0f47791eb862' # ID of Starbucks (星巴克)
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4bf2301b189f0f47791eb862?client_id=JRMAOQ23QH2AR2G01QNCFARI5IKTISMVT2PHM3D0DXKGHF5N&client_secret=M1P2CGJKPR0J1N1GMUV0P5CH1BGLFD3SHIPP4YTOFQ4NTATE&v=20190520'

In [24]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto'])


{'id': '4bf2301b189f0f47791eb862',
 'name': 'Starbucks (星巴克)',
 'contact': {'phone': '+862163552271',
  'formattedPhone': '+86 21 6355 2271',
  'twitter': 'starbucks',
  'instagram': 'starbucks',
  'facebook': '22092443056',
  'facebookUsername': 'Starbucks',
  'facebookName': 'Starbucks'},
 'location': {'address': '豫园路100号 | 100 Yuyuan Rd.',
  'crossStreet': '豫园 | Yu Garden',
  'lat': 31.228536577839012,
  'lng': 121.48721181905799,
  'labeledLatLngs': [{'label': 'display',
    'lat': 31.228536577839012,
    'lng': 121.48721181905799}],
  'postalCode': '200010',
  'cc': 'CN',
  'city': '上海市',
  'state': '上海市',
  'country': '中国',
  'formattedAddress': ['豫园路100号 | 100 Yuyuan Rd. (豫园 | Yu Garden)',
   '上海市',
   '上海市, 200010',
   '中国']},
 'canonicalUrl': 'https://foursquare.com/v/starbucks-%E6%98%9F%E5%B7%B4%E5%85%8B/4bf2301b189f0f47791eb862',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'ic

In [25]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.6


In [26]:
venue_id = '50b70a92e4b055ce10fff6bf' # ID of Costa Coffee (咖世家)
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/50b70a92e4b055ce10fff6bf?client_id=JRMAOQ23QH2AR2G01QNCFARI5IKTISMVT2PHM3D0DXKGHF5N&client_secret=M1P2CGJKPR0J1N1GMUV0P5CH1BGLFD3SHIPP4YTOFQ4NTATE&v=20190520'

In [27]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto'])


{'id': '50b70a92e4b055ce10fff6bf',
 'name': 'Costa Coffee (咖世家)',
 'contact': {'twitter': 'costacoffee',
  'facebook': '124310790964800',
  'facebookUsername': 'CostaCoffee',
  'facebookName': 'Costa Coffee'},
 'location': {'address': '168 Zhoujin Rd | 昼锦路168号',
  'crossStreet': 'Yu Fashion Garden | 豫城时尚',
  'lat': 31.226842885699735,
  'lng': 121.48780706417091,
  'labeledLatLngs': [{'label': 'display',
    'lat': 31.226842885699735,
    'lng': 121.48780706417091}],
  'postalCode': '200085',
  'cc': 'CN',
  'neighborhood': 'Yùyuán',
  'city': 'Huangpu',
  'state': '上海市',
  'country': '中国',
  'formattedAddress': ['168 Zhoujin Rd | 昼锦路168号 (Yu Fashion Garden | 豫城时尚), Yùyuán',
   'Huangpu',
   '上海市, 200085',
   '中国']},
 'canonicalUrl': 'https://foursquare.com/v/costa-coffee-%E5%92%96%E4%B8%96%E5%AE%B6/50b70a92e4b055ce10fff6bf',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix':

In [28]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.2


In [32]:
venue_id = '5b3da1542a7ab60039b2571f' # ID of Luckin Coffee (瑞幸咖啡)
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/5b3da1542a7ab60039b2571f?client_id=JRMAOQ23QH2AR2G01QNCFARI5IKTISMVT2PHM3D0DXKGHF5N&client_secret=M1P2CGJKPR0J1N1GMUV0P5CH1BGLFD3SHIPP4YTOFQ4NTATE&v=20190520'

In [33]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto'])


{'id': '5b3da1542a7ab60039b2571f',
 'name': 'Luckin Coffee (瑞幸咖啡)',
 'contact': {},
 'location': {'address': "No.10 Pu'an Road",
  'lat': 31.226364,
  'lng': 121.47625,
  'labeledLatLngs': [{'label': 'display', 'lat': 31.226364, 'lng': 121.47625}],
  'cc': 'CN',
  'city': '上海市',
  'state': '上海市',
  'country': '中国',
  'formattedAddress': ["No.10 Pu'an Road", '上海市', '上海市', '中国']},
 'canonicalUrl': 'https://foursquare.com/v/luckin-coffee-%E7%91%9E%E5%B9%B8%E5%92%96%E5%95%A1/5b3da1542a7ab60039b2571f',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '¥'},
 'likes': {'count': 0, 'groups': []},
 'dislike': False,
 'ok': False,
 'allowMenuUrlEdit': True,
 'beenHere': {'count': 0,

In [38]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


## Looks like in this area, the highest rating goes to a Costa, with a mark of 6.2; others either don't have a rating or just below what Costa has, for example, the second highest rated shop is a Starbucks with a mark of 5.6; and our newly emerged coffee force in China, Lucking coffee, has no ratings yet till this moment, this is perhaps the mainstream public opinion social medium platform is not Foursquare, but others such as "Da Zhong Dian ping (大众点评)".

## Try to see tips that left by users, but found nothing, the reason is the same as above about no ratings with Lucking Coffee: that Foursquare is not a mainstream platform in China

In [39]:
result['response']['venue']['tips']['count']

0

In [40]:
## Costa Tips
venue_id = '50c4614ce4b0b83e09a83a90' # ID of Costa
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ce5fe22db04f52f5f1ed317'},
 'response': {'tips': {'count': 0, 'items': []}}}

In [44]:
tips = results['response']['tips']['items']

In [45]:
tips

[]